# Introduction to LangChain v0.2.0 and LCEL: LangChain Powered RAG

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

In the notebook, you'll complete the following Tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Initialize a Simple Chain using LCEL
  4. Implement Naive RAG using LCEL
  
Let's get started!



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [1]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [2]:
!pip install -qU qdrant-client

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [3]:
!pip install -qU tiktoken pymupdf

## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-4o` model.

In [5]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

####❓ Question #1:

What other models could we use, and how would the above code change?

> HINT: Check out [this page](https://platform.openai.com/docs/models) to find the answer!

---
####  ANSWER #1:
We could use gpt-4o-mini or gpt-3.5 or gpt-4.  Code change is simply changing the model parameter above.

---


### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
system_template = """ You are a legendary and mythical Wizard. You speak in riddles 
and make obscure and pun-filled references to exotic cheeses.
"""
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.
- `openai_chat_model`, which is a LangChain Runnable wrapped OpenAI client.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [8]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







#### Mike Dean Comment

I need to dissect this.  Have added a couple cells to invoke each piece separately to see if I completely understand.

In [12]:
print(chat_prompt.invoke({"content": "Hello world!"}))

messages=[SystemMessage(content=' You are a legendary and mythical Wizard. You speak in riddles \nand make obscure and pun-filled references to exotic cheeses.\n'), HumanMessage(content='Hello world!')]


In [22]:
experimentalPrompt = chat_prompt.invoke({"content": "Hello world!"})
print(openai_chat_model.invoke(experimentalPrompt))

content='Ah, greetings, seeker of knowledge! You have but scratched the rind of the cosmos. Tell me, do you seek wisdom that melts like a fine Camembert, or perhaps a hint sharper, like the aged Cheddar of destiny?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 39, 'total_tokens': 87}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None} id='run-759f8a58-df84-40b0-b55f-574131373d26-0' usage_metadata={'input_tokens': 39, 'output_tokens': 48, 'total_tokens': 87}


In [26]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, greetings, seeker of knowledge! You have stirred the cauldron of curiosity. Tell me, do you seek the wisdom of the ages, or merely a slice of enlightenment, as rich as a wheel of Camembert?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 39, 'total_tokens': 86}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None} id='run-00891f4d-5e31-493f-957f-0757b23d526e-0' usage_metadata={'input_tokens': 39, 'output_tokens': 47, 'total_tokens': 86}


Let's try it out with a different prompt!

In [27]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content='Ah, thou seekest the path to Pythonic power, do you? Very well, listen closely, for the journey is like a fine wheel of Gouda: it takes time, patience, and a bit of aging to achieve perfection.\n\n1. **Savor the Syntax**: Just as a connoisseur savors a slice of Roquefort, so must you savor the syntax of Python. Immerse thyself in the language’s core principles and let the simplicity and readability melt upon thy tongue.\n\n2. **Script and Slice**: Practice, dear seeker, is the curd that forms the cheese. Write scripts daily, exploring new libraries and modules. Slice through challenges and let your code mature.\n\n3. **The Wisdom of Elders (Documentation)**: Seek the wisdom of the ancient scrolls, the Python documentation. In those hallowed texts, you’ll find treasures and secrets akin to a hidden cave of Parmigiano-Reggiano.\n\n4. **Join the Circle of Coders**: Many moons have shone upon the Earth, and many Pythonistas gather in forums and communities. Engage with t

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [35]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content="LangChain is a framework designed specifically for developing applications powered by large language models (LLMs). It provides modular components and tools to streamline the creation of various types of applications, such as chatbots, Generative Question-Answering (GQA) systems, summarization tools, and more.\n\nLangChain is built around several key concepts:\n\n1. **LLM Wrappers**: These are interfaces that connect to different large language models, allowing developers to leverage the capabilities of various LLMs without needing to manage the underlying infrastructure.\n\n2. **Prompt Management**: LangChain facilitates the creation, formatting, and optimization of prompts, which are critical for guiding the behavior of LLMs effectively.\n\n3. **Chains**: These are sequences of calls to LLMs or other utilities, which can include conditionals, loops, and other control structures. Chains allow for the construction of complex workflows that can process input and generate the de

Well, that's not very good - is it!

#### Mike Dean Response
Turns out it was pretty good - probably because we switched to GPT-4o instead of GPT-3.5.
LangChain would have been inside the training period (up to October 2023 compared with
GPT-3.5 ended September 2021).

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [36]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

content='LangChain Expression Language (LECL) is a domain-specific language designed to facilitate the chaining and orchestration of various language model operations, data transformations, and integrations within the LangChain framework. LangChain is a toolkit for building applications with large language models (LLMs), and LECL provides a structured way to define and manage these complex workflows.\n\nWith LECL, developers can specify sequences of operations, such as querying a language model, processing its output, and integrating with other data sources or APIs, all in a concise and human-readable format. This helps in creating more robust, maintainable, and scalable applications that leverage the power of LLMs.\n\nKey features of LECL might include:\n\n1. **Modularity**: Breaking down complex tasks into reusable components.\n2. **Interoperability**: Seamless integration with different APIs and data sources.\n3. **Ease of Use**: Simplified syntax for defining intricate workflows.\n


However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [46]:
HUMAN_TEMPLATE = """ 
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided 
context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language (LCEL) is a declarative way to easily compose chains together in LangChain. It offers several benefits, including automatic support for sync, async, batch, and streaming interfaces, the ability to handle errors gracefully with fallbacks, automatic parallel execution of components that can run in parallel, and seamless integration with LangSmith for detailed tracing and observability of complex chains.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 275, 'total_tokens': 353}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None} id='run-7fe74582-407f-4adc-a3f9-03d849f5db73-0' usage_metadata={'input_tokens': 275, 'output_tokens': 78, 'total_tokens': 353}


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

> NOTE: It is recommended you do not run the following cells, they are purely for demonstrative purposes.

In [ ]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [ ]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
len(enc.encode(context))

636144

The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-128k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [55]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [60]:
chunks = text_splitter.split_text(CONTEXT)

In [61]:
len(chunks)

3

In [62]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at ev

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

####🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

---
Brainstorm some ideas that would split large single documents into smaller documents.

1. I have a use case that consists of a Federal repository of data element definitions, which I download as a JSON document (85Mb) and I parse this to pull the relevant portions, writing each data element into a separate row of a csv file.  If I use a CSVLoader from LangChain, it creates separate documents for each row and each document is sufficiently small that I don't need to do a further split.  So, if a large document can be converted to smaller documents without a splitter, then you can simply vectorize each document.
2. If you have a structured document (for example, a document written in LaTeX or that has Markdown tags), then you could enhance the recursive splitting by using information about headers to be the high level split, instead of '\n\n'.
3. Small chunks have more semantic specificity, but much less semantic sensitivity.  Suppose I have a list of twenty criteria for a patient to be enrolled in a clinical trial, spanning two pages.  If I chunk large enough to have those two pages in the same chunk, then if I query for inclusion criteria I will get the whole list.  If I chunk smaller, then some of the chunks won't have any semantic connection to "inclusion criteria" because that was only present in the initial chunk.  SO we could use a strategy of using small chunks but somehow attaching metadata to those chunks to point to larger documents, and then feed those larger documents into the context.  So do the query, hit at least one semantically close chunk, point to two pages that include the whole list of criteria, and feed the two pages to the LLM.
---

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using OpenAI's `text-embedding-3` model as our embedding model today!

Let's load it up through LangChain.

In [63]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

#### ❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

#### ANSWER: 1536

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [64]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [70]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print(v[:10])
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 1536
[0.0005607009516097605, -0.0366472490131855, 0.023857517167925835, -0.008668888360261917, 0.050974905490875244, 0.03672611713409424, -0.006112256553024054, -0.01965123414993286, -0.023528901860117912, 0.011560709215700626]



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes importan

Okay, great. Let's create a query - and then embed it!

In [72]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")
print(query_vector[:5])


Vector of Size: 1536
[0.0009704043623059988, -0.034166883677244186, 0.022720256820321083, -0.03589685261249542, 0.021264199167490005]


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [74]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [77]:

max_similarity = -float('inf')
closest_chunk = ""  

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.5372984870519116


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [78]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [79]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [80]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?', 'response': AIMessage(content='Yes, LCEL can help take code from the notebook to production. Since any chain constructed with LCEL will automatically have full sync, async, batch, and streaming support, it makes it easy to prototype a chain in a Jupyter notebook using the sync interface and then expose it as an async streaming interface for production use.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 153, 'total_tokens': 218}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-07cfe5d4-7966-4d0f-940c-f36abcc87263-0', usage_metadata={'input_tokens': 153, 'output_tokens': 65, 'total_tokens': 218}), 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as 

####❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use your newly created `simple_rag` to help you answer this question!

In [84]:
print(simple_rag("What does LCEL do that makes it more reliable at scale?",
           embeddings_dict, embedding_model, chat_chain))

{'query': 'What does LCEL do that makes it more reliable at scale?', 'response': AIMessage(content='LCEL makes it more reliable at scale by providing full sync, async, batch, and streaming support for any chain constructed this way. This comprehensive support allows for easy prototyping and seamless transition to different execution modes, ensuring that the chains can handle various demands and workloads efficiently.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 154, 'total_tokens': 209}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-abc641f5-f2f8-491b-9ac6-997e6b214d92-0', usage_metadata={'input_tokens': 154, 'output_tokens': 55, 'total_tokens': 209}), 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed 

---
#### ANSWER 3 from simple_rag:
LCEL makes it more reliable at scale by providing full sync, async, batch, and streaming support for any chain constructed this way. This comprehensive support allows for easy prototyping and seamless transition to different execution modes, ensuring that the chains can handle various demands and workloads efficiently.

---